In [2]:
import pandas as pd

file = "./data/startup data.csv"  

df=pd.read_csv(file)
#df["is_CA"]

In [7]:
# Drop specified columns
df = df.drop(['Unnamed: 0', 'zip_code', 'id', 'Unnamed: 6', 'labels', 'state_code.1', 'object_id','closed_at','age_last_milestone_year','first_funding_at','last_funding_at','age_first_milestone_year'], axis=1)

# Create a new column 'state' based on the existing state columns
df['state'] = df[['is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate']].idxmax(axis=1)
df['state'] = df['state'].apply(lambda x: x[3:] if x != 'is_otherstate' else 'Other')

# Drop the individual state columnss
df = df.drop(['is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate'], axis=1)

# Create a new column 'Sector' based on the individual sector columns
sectors = ['is_software', 'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 
           'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 'is_othercategory']

df['Sector'] = df[sectors].idxmax(axis=1)

# Mapping to rename sectors
sector_mapping = {
    'is_software': 'software',
    'is_web': 'web',
    'is_mobile': 'mobile',
    'is_enterprise': 'enterprise',
    'is_advertising': 'advertising',
    'is_gamesvideo': 'gamesvideo',
    'is_ecommerce': 'ecommerce',
    'is_biotech': 'biotech',
    'is_consulting': 'consulting',
    'is_othercategory': 'othercategory'
}

# Rename sectors using mapping
df['Sector'] = df['Sector'].replace(sector_mapping)

# Now 'is_software' becomes 'software', 'is_web' becomes 'web', and so on.

# Drop the individual sector columns
df = df.drop(sectors, axis=1)



In [8]:
#cellule pour tester des trucs
df.head()
column_names = df.columns

# Print the column names
print(column_names)

Index(['state_code', 'latitude', 'longitude', 'city', 'name', 'founded_at',
       'age_first_funding_year', 'age_last_funding_year', 'relationships',
       'funding_rounds', 'funding_total_usd', 'milestones', 'category_code',
       'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC',
       'has_roundD', 'avg_participants', 'is_top500', 'status', 'state',
       'Sector'],
      dtype='object')


In [33]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

# Assuming 'df' is your DataFrame with columns including 'status' and 'category_code'
# Replace this with your actual data

# Sample data for demonstration

# Calculate the proportion of acquired companies in each category
category_proportions = df.groupby('Sector')['status'].apply(lambda x: (x == 'acquired').mean()).reset_index(name='proportion')


# Créer une nouvelle colonne 'new_funding_total_usd' avec les valeurs modifiées
df['age_first_funding'] = df['age_first_funding_year'].apply(lambda x: x if x >=0 else None)


# Create a Dash app
app = dash.Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Define the layout
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(children='How start-up succeed in America',
            style={'text-align': 'center', 'color': colors['text'], 'font-family': 'Arial, sans-serif'}),

    dbc.Row([
        dbc.Col([dcc.Graph(
            id='bar-chart',
            figure=px.bar(category_proportions, x='Sector', y='proportion',
                     color='Sector',
                     title='Proportion of Acquired Companies by Sector',
                     ))
        ]),
        dbc.Col([dcc.Graph(
            id='histogram',
            figure=px.histogram(df,x='age_last_funding_year'))
        ])
    ])
])


app.run_server(debug=True)


In [20]:
px.box(df,x='funding_total_usd')
px.histogram(df, x='funding_total_usd', title='Histogramme du Financement Total'))

In [2]:
import folium 
import pandas as pd
import geopandas as gpd

map = folium.Map(location= [37.0902, -95.7129 ], zoom_start =3.5 )

map
